In [2]:
# conda install -c conda-forge python-annoy
# conda install -c conda-forge pymorphy2
# conda install -c conda-forge stop-words
# conda install -c conda-forge gensim
# conda install -c conda-forge ipywidgets

# pip install chardet для определения кодировщика

In [5]:
import os
import string
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import codecs
DIR="..\\..\\chunk\\14\\"

In [49]:
pds=pd.read_csv(f'{DIR}ProductsDataset.csv')


In [50]:
pds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35548 entries, 0 to 35547
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           35548 non-null  object 
 1   descrirption    33537 non-null  object 
 2   product_id      35536 non-null  object 
 3   category_id     35536 non-null  float64
 4   subcategory_id  35536 non-null  object 
 5   properties      35536 non-null  object 
 6   image_links     35533 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [51]:
pds[pds.category_id==22].head(5)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
14,"Ветровка детская, Италия",Практичная и стильная ветровка Club des Sports...,593a439ec6ab9e8bb5071895,22.0,2204,"{'detskie_tzvet': 'Черный', 'detskie_razmer_ro...",http://cache3.youla.io/files/images/360_360/59...
18,Розовое платье с пелеринкой,Платье для девочки (9-10 лет) в отличном состо...,596d0169de88542768771815,22.0,2211,"{'detskie_razmer_rost': '135-140 (10 лет)', 'd...",http://cache3.youla.io/files/images/360_360/59...
20,Комбинезон 68р на девочку,"Фирма C&A, привозили из Германии, отличное кач...",57de45b50cc3da643066dafc,22.0,2204,"{'detskie_pol': 'Девочкам', 'detskie_razmer_ro...",http://cache3.youla.io/files/images/360_360/57...


вспомогательный обработчик текстов, включающий функции удаления стопслов, знаков пунктуации и приведения слов к начальной форме
для уменьшения размерности корпуса векторов, сопоставляемых набору разных слов

In [31]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

Предобработаем ответы mail.ru из файла: выбрать только вопросы и поместить в датасет.
эти вопросы предполагается использовать для обучения логистической регрессии - как тексты, не связанные с товарными запросами(хотя не исключено, что в запросах мейл такие могут присутствовать)

In [53]:
# %%time
# from tqdm.notebook import tqdm
# tqdm.pandas()

from tqdm import tqdm, notebook
import time
df=pd.DataFrame(columns=['question','answer'])

#Мы идем по всем записям, просматриваем содержимое
# и сразу после знака --- находим вопрос
with open(file_name:=f"{DIR}Otvety.txt", "r",encoding='utf-8') as fin:
    t = tqdm(total=os.stat(file_name).st_size) # инициализировать прогресбар размером файла
    start_time = time.time() #использовать интервал для настройки частоты отображения прогресбара
    question = False
    written = False
    lastpos=0
    tmpcnt=0
    while line:=fin.readline():
        if time.time()-start_time>2:
            start_time = time.time()
            t.update((fin.tell()-lastpos,lastpos:=fin.tell())[0])
        # if tmpcnt>10: break
        if line.startswith("---"):#начало блока вопрос/несколько ответов
            tmpcnt+=1
            written = False
            question = False
            continue
        line=line.replace("\t", " ").replace("<br>", "").strip().lower()
        if question:
            # сохранить ответ в текущий(последний) ряд
            df.iloc[-1]['answer']=df.iloc[-1]['answer']+line +'\t'
            written = True
            continue
        if not question:
            # сохранить вопрос
            question = True
            df=pd.concat([df,pd.DataFrame({'question':[line],'answer':['']})],ignore_index=True)
            continue
    t.update((fin.tell()-lastpos,lastpos:=fin.tell())[0])

display(df)


 36%|███▌      | 658252978/1847850502 [2:26:26<7:43:42, 42756.26it/s] 

In [52]:
print(df.iloc[3]['answer'].split('\t'))

['если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!.', 'что не знаешь что делать? тогда отдай тому кто знает, а если серьезно- можешь отнести их в полицию они найдут что с ними сделать, или другой вариант возьми чуть, а остальные надежно спрячь пускай лежат будешь брать сколько необходимо чтобы подозрений не вызывало не у кого от куда у тебя такие деньги..', 'а не скажете местечко, где такие находки попадаются?))) если бы я нашла два миллиона, я прежде всего растерялась и преб

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [7]:
sentences = []

c = 0

with open(f"{DIR}Otvety.txt", "r",encoding='utf-8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_1596\871568580.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [8]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

In [27]:
sentences[:14]

[['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво'],
 ['советовать', 'сися', 'завести'],
 ['слава', 'бог', 'мужик', 'завести', 'советовать', 'сватать', 'сватать'],
 ['советовать', 'завести', 'любовник', 'муж', 'кошка', 'brприветик'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой'],
 ['вобщий', 'прикалывать', 'тема'],
 ['этобыть',
  'редкость',
  'забавно',
  'знать',
  'фальшивка',
  'прикольно',
  'силиконовый',
  'сиська',
  'налепить',
  'синтетический',
  'волос'],
 ['делать', 'найти', '2', 'миллион', 'рубль'],
 ['счастие',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'нарваться',
  'бабушка',
  'помочий',
  'внук',
  'покупка',
  'квартира',
  'бандит',
  'разг

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [10]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open(f"{DIR}prepared_answers.txt", "r",encoding = "utf-8") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_1596\3324386442.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

In [ ]:
with open(f"{DIR}prepared_answers1.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        print(spls[1])
        # index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        print(preprocess_txt(spls[0]))
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        print(vector)
        # index.add_item(counter, vector)


Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [32]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [31]:
print(index.get_nns_by_vector.__doc__)

Returns the `n` closest items to vector `vector`.

:param search_k: the query will inspect up to `search_k` nodes.
`search_k` gives you a run-time tradeoff between better accuracy and speed.
`search_k` defaults to `n_trees * n` if not provided.

:param include_distances: If `True`, this function will return a
2 element tuple of lists. The first list contains the `n` closest items.
The second list contains the corresponding distances.


In [33]:
find_answer('вопрос тдв отдыхать хорошо любовник советовать?')

'И за рулём и на руле, лишь бы спать не мешали..... \n'

In [19]:
index.get_n_trees()

10

In [20]:
index.get_n_items()

1163342